In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install "git+https://github.com/albumentations-team/albumentations.git"

In [1]:
import sys
sys.path.append("/content/drive/MyDrive/Kaggle/Sartorius")

In [2]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer
from albumentations import *
import torch
import os
from detectron2.data import detection_utils
from utils.aug import MyMapper
from detectron2.solver.build import *
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import warnings
warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
class Config:
  pixel_mean = [128,128,128]
  pixel_std = [13.235,13.235,13.235]
  anchor_generators_sizes = [[8], [16], [32], [64],[128]]
  anchor_generators_aspect_ratios = [[0.5, 1.0, 2.0]]

In [4]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=1) == 0  # Extra objects
    false_negatives = np.sum(matches, axis=0) == 0  # Missed objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    # @classmethod
    # def build_train_loader(cls, cfg, sampler=None):
    #     return build_detection_train_loader(
    #         cfg, mapper=MyMapper(cfg), sampler=sampler
    #     )

    def build_hooks(self):
      # copy of cfg
      cfg = self.cfg.clone()

      # build the original model hooks
      hooks = super().build_hooks()

      # add the best checkpointer hook
      hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
                                        DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                                        "MaP IoU",
                                        "max",
                                        ))
      return hooks

    # @classmethod
    # def build_optimizer(cls,cfg,model) -> torch.optim.Optimizer:
    #   """
    #   Build an optimizer from config.
    #   """
    #   params = get_default_optimizer_params(
    #       model,
    #       base_lr=cfg.SOLVER.BASE_LR,
    #       weight_decay_norm=cfg.SOLVER.WEIGHT_DECAY_NORM,
    #       bias_lr_factor=cfg.SOLVER.BIAS_LR_FACTOR,
    #       weight_decay_bias=cfg.SOLVER.WEIGHT_DECAY_BIAS,
    #   )
    #   return maybe_add_gradient_clipping(cfg, torch.optim.SGD)(
    #       params,
    #       lr=cfg.SOLVER.BASE_LR,
    #       momentum=cfg.SOLVER.MOMENTUM,
    #       nesterov=cfg.SOLVER.NESTEROV,
    #       weight_decay=cfg.SOLVER.WEIGHT_DECAY,
    #   )

    # @classmethod
    # def build_lr_scheduler(cls,cfg, optimizer) -> torch.optim.lr_scheduler._LRScheduler:
    #   """
    #   Build a LR scheduler from config.
    #   """
    #   name = cfg.SOLVER.LR_SCHEDULER_NAME

    #   if name == "WarmupMultiStepLR":
    #       steps = [x for x in cfg.SOLVER.STEPS if x <= cfg.SOLVER.MAX_ITER]
    #       if len(steps) != len(cfg.SOLVER.STEPS):
    #           logger = logging.getLogger(__name__)
    #           logger.warning(
    #               "SOLVER.STEPS contains values larger than SOLVER.MAX_ITER. "
    #               "These values will be ignored."
    #           )
    #       sched = MultiStepParamScheduler(
    #           values=[cfg.SOLVER.GAMMA ** k for k in range(len(steps) + 1)],
    #           milestones=steps,
    #           num_updates=cfg.SOLVER.MAX_ITER,
    #       )
    #   elif name == "WarmupCosineLR":
    #       sched = CosineParamScheduler(1, 1/40)
    #   elif name == "CyclicLR":
    #       sched = torch.optim.lr_scheduler.CyclicLR(optimizer,
    #                                                 base_lr=cfg.SOLVER.BASE_LR, max_lr=5*cfg.SOLVER.BASE_LR,
    #                                                 base_momentum=0.85, max_momentum=0.95)

    #   else:
    #       raise ValueError("Unknown LR scheduler: {}".format(name))

    #   sched = WarmupParamScheduler(
    #       sched,
    #       cfg.SOLVER.WARMUP_FACTOR,
    #       min(cfg.SOLVER.WARMUP_ITERS / cfg.SOLVER.MAX_ITER, 1.0),
    #       cfg.SOLVER.WARMUP_METHOD,
    #   )
    #   return LRMultiplier(optimizer, multiplier=sched, max_iter=cfg.SOLVER.MAX_ITER)

In [5]:
def run():
  dataDir = "/content/drive/MyDrive/Kaggle/Sartorius/input/sartorius-cell-instance-segmentation"
  DatasetCatalog.clear()
  MetadataCatalog.clear()
  register_coco_instances(f'sartorius_train',{}, '/content/drive/MyDrive/Kaggle/Sartorius/input/all/annotations_train.json'.format(fold), dataDir)
  register_coco_instances(f'sartorius_val',{},'/content/drive/MyDrive/Kaggle/Sartorius/input/all/annotations_val.json'.format(fold), dataDir)

  cfg = get_cfg()
  cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
  cfg.DATASETS.TRAIN = (f"sartorius_train",)
  cfg.DATASETS.TEST = (f"sartorius_val",)
  # cfg.MODEL.PIXEL_MEAN = Config.pixel_mean
  # cfg.MODEL.PIXEL_STD = Config.pixel_std

  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/livecell_pretrained/pretrained_by_livecell_324LB.pth"#"/content/drive/MyDrive/Kaggle/Sartorius/model/pretrained/LIVECell_anchor_based_model.pth"
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.SOLVER.BASE_LR = 1e-3
  #cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
  cfg.SOLVER.MAX_ITER = 10000 #len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH * 20 #尝试20ep 用lr调整
  cfg.SOLVER.STEPS = (5000, 7500, 9000)
  #cfg.SOLVER.GAMMA = 0.2
  cfg.INPUT.MIN_SIZE_TRAIN = (780, 800)
  cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
  cfg.INPUT.MIN_SIZE_TEST = 1040
  cfg.INPUT.MAX_SIZE_TEST = 1408
  cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 256
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
  #cfg.TEST.EVAL_PERIOD = 
  cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get(f"sartorius_train")) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

  cfg.INPUT.MASK_FORMAT='bitmask'
  cfg.TEST.DETECTIONS_PER_IMAGE = 1000
  cfg.SOLVER.AMP.ENABLED  = False

  # cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = Config.anchor_generators_aspect_ratios
  # cfg.MODEL.ANCHOR_GENERATOR.SIZES = Config.anchor_generators_sizes
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]
  # cfg.MODEL.RESNETS.OUT_FEATURES = ["res2", "res3", "res4", "res5"]
  # cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
  # #cfg.MODEL.FPN.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IN_FEATURES = ["p2", "p3", "p4", "p5","p6"]
  # cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5", "p6"]

  # cfg.MODEL.RESNETS.DEFORM_ON_PER_STAGE = [False, True, True, True] # on Res3,Res4,Res5
  # cfg.MODEL.RESNETS.DEFORM_MODULATED = True
  # cfg.MODEL.RESNETS.DEFORM_NUM_GROUPS = 2
  # cfg.MODEL.RESNETS.NORM = "GN"
  # cfg.MODEL.ROI_BOX_HEAD.NAME = "FastRCNNConvFCHead"
  # cfg.MODEL.ROI_BOX_HEAD.NUM_CONV = 4
  # cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 1
  # cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"  
  # cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION = 7
  # cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True  
  # cfg.MODEL.ROI_MASK_HEAD.NUM_CONV = 8
  # cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"
  # cfg.MODEL.ROI_HEADS.IOU_THRESHOLDS = [0.5]
  # cfg.MODEL.ROI_BOX_CASCADE_HEAD.IOUS = (0.5, 0.6, 0.7)

  # cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 3000
  # cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 3000
  
    
  # cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = "ciou"
  # cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = "ciou"
  # cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_WEIGHT = 10.0
  #cfg.SOLVER.AMP.ENABLED = True

  cfg.OUTPUT_DIR = "/content/drive/MyDrive/Kaggle/Sartorius/model/finetuned/livecell_cascade_v0/".format(fold)

  os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
  trainer = Trainer(cfg) 
  trainer.resume_or_load(resume=False)
  trainer.train()

In [ ]:
# 2 的理论score最高
for fold in range(1,2):
  run()

#run_all()

[12/17 18:55:35 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.0.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.0.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.1.cls_score.bias' to the model due to incompatible shapes: (9,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.2.cls_score.weight' to the model due to incompatible shapes: (9, 1024) in the checkpoint but (4, 1024) in th

[12/17 18:55:41 d2.engine.train_loop]: Starting training from iteration 0
[12/17 18:57:16 d2.utils.events]:  eta: 12:41:55  iter: 19  total_loss: 6.798  loss_cls_stage0: 1.343  loss_box_reg_stage0: 0.3832  loss_cls_stage1: 1.437  loss_box_reg_stage1: 0.5  loss_cls_stage2: 1.336  loss_box_reg_stage2: 0.4213  loss_mask: 0.6903  loss_rpn_cls: 0.2655  loss_rpn_loc: 0.2938  time: 4.6417  data_time: 0.1041  lr: 1.9981e-05  max_mem: 11730M
[12/17 18:58:47 d2.utils.events]:  eta: 12:40:09  iter: 39  total_loss: 5.391  loss_cls_stage0: 1.046  loss_box_reg_stage0: 0.3242  loss_cls_stage1: 1.092  loss_box_reg_stage1: 0.4501  loss_cls_stage2: 0.9894  loss_box_reg_stage2: 0.3709  loss_mask: 0.6609  loss_rpn_cls: 0.2206  loss_rpn_loc: 0.2816  time: 4.6131  data_time: 0.0429  lr: 3.9961e-05  max_mem: 11730M
[12/17 19:00:19 d2.utils.events]:  eta: 12:39:39  iter: 59  total_loss: 4.278  loss_cls_stage0: 0.6722  loss_box_reg_stage0: 0.2995  loss_cls_stage1: 0.6963  loss_box_reg_stage1: 0.3949  loss_cls_